# Import Libraries

In [1]:
import sys
import redpitaya_scpi as scpi
import numpy as np
import time
import matplotlib.pyplot as plt

import IPython
import ipywidgets as widgets

from bokeh.plotting import figure, show
from bokeh.io import output_notebook, push_notebook
from bokeh.layouts import column
from bokeh.models import ColumnDataSource, Toggle, Range1d

# Configure instruments and libraries

In [2]:
# Red Pitaya Stuff

address = "rp-f06897.local"
rp = scpi.scpi(address)
dec_exp = np.array([0,3,6,10,13,16])
dec = 2**dec_exp
t_total = np.array([131.072e-6, 1.049e-3, 8.389e-3, 134.218e-3,1.074, 8.590])

# For the gui updates during the live view loops
ipython = IPython.get_ipython()

# For Bokeh
output_notebook()

Loading BokehJS ...

# Function for reading IV data

MrSQUID hardware conversion factors:

* Y-axis: 10000 V/V (1V on Red Pitaya = 100 uV across SQUID)
* X-axis: 10000 V/A (1V on Red Pitaya = 100 uA through SQUID)

Max ranges:

* Current monitor: +/- 2.2V = +/- 220 uA?
* Voltage readout: +/- 8.8V = +/- 880 uV?

To do: 

* Use timestamps to figure out where we are losing time
* See if switching to binary read, for example, would make this function run faster

Notes: 

With `timebase = -3`, the trace is 134 microseconds long or so. But triggering and acquiring the trace with the code below takes 1.8 seconds to bring the 134 microsecond trace into python! This is a very slow update rate.

In [44]:
def get_IV_t(timebase=-3):
    tn = [time.time()]
    
    # Configure timebase and input range settings
    rp.tx_txt("ACQ:DEC %d" % dec[timebase])
    #print("setting dec to %d " % dec[timebase])
    #print("t_total is %f" % t_total[timebase])
    rp.tx_txt('ACQ:SOUR1:GAIN HV')
    rp.tx_txt('ACQ:SOUR2:GAIN HV')
    
    # Start the acquisition
    rp.tx_txt('ACQ:START')
    time.sleep(t_total[timebase]+0.1) # to avoid getting old data in the buffer
    rp.tx_txt('ACQ:TRIG NOW')
    
    tn.append(time.time())
    
    # Wait until it's done
    while 1:
        rp.tx_txt('ACQ:TRIG:STAT?')
        if rp.rx_txt() == 'TD':
            break
    
    tn.append(time.time())
    
    # Download the traces
    rp.tx_txt('ACQ:SOUR1:DATA?')
    buff_string = rp.rx_txt()
    buff_string = buff_string.strip('{}\n\r').replace("  ", "").split(',')
    I = np.array(list(map(float, buff_string))) / 1e4
    
    tn.append(time.time())
    
    rp.tx_txt('ACQ:SOUR2:DATA?')
    buff_string = rp.rx_txt()
    buff_string = buff_string.strip('{}\n\r').replace("  ", "").split(',')
    V = np.array(list(map(float, buff_string))) / 1e4
    
    tn.append(time.time())
    
    # Create the appropriate time array
    t = np.linspace(0,t_total[timebase],len(I))
    
    print("Total time: %f" % (tn[-1]-tn[0]))
    print("Splits:")
    for i in range(len(tn)-1):
        print("%f" % (tn[i+1]-tn[i]))
    # Return the traces
    return I,V,t

In [45]:
get_IV_t();

Total time: 1.773177
Splits:
0.235024
0.403040
0.585059
0.550055


# IV vs Time Live Display

In [19]:
# Shit, spent about 2 hours on this! We need to create the ranges first and pass them to the 
# figure initialisation if we want to be able to change them...

def update_display():
    I,V,t = get_IV_t()
    I *= 1e6
    V *= 1e6
    I_avg = np.average(I)
    I_ptp = np.max(I) - np.min(I)
    p1.title.text = "Current          Average = %.2f uA     Peak-to-Peak %.2f uA" % (I_avg, I_ptp)
    V_avg = np.average(V)
    V_ptp = np.max(V) - np.min(V)
    p2.title.text = "Voltage          Average = %.2f uV     Peak-to-Peak %.2f uV" % (V_avg, V_ptp)
    R_ptp.value = "%.2f" % (V_ptp / I_ptp)
    R_avg.value = "%.2f" % (V_avg / I_avg)
    source1.data = dict(x=t, y=I)
    source2.data = dict(x=t, y=V)
    if scale_mode.value == "Autoscale":
        p1.y_range.start = np.min(I)-I_ptp*0.1
        p1.y_range.end = np.max(I)+I_ptp*0.1
        p2.y_range.start = np.min(V)-V_ptp*0.1
        p2.y_range.end = np.max(V)+V_ptp*0.1
    else:
        p1.y_range.start = -p1_full_scale
        p1.y_range.end = p1_full_scale
        p2.y_range.start = -p2_full_scale
        p2.y_range.end = p2_full_scale
        
range1 = Range1d()
range2 = Range1d()
source1 = ColumnDataSource()
source2 = ColumnDataSource()

p1 = figure(title="Current", plot_height=300, plot_width=900,toolbar_location=None, y_range = range1)
p1.yaxis.axis_label = 'Curret (uA)'

p2 = figure(title="Voltage", plot_height=300, plot_width=900,toolbar_location=None, y_range = range2)
p2.xaxis.axis_label = 'Time (s)'
p2.yaxis.axis_label = 'Voltage (uV)'

p1.line('x', 'y', source=source1)
p2.line('x', 'y', source=source2)

p1_full_scale = 240 #uA
p2_full_scale = 800 # uV

stop_button = widgets.ToggleButton(description='Stop')
scale_mode = widgets.RadioButtons(options=['Autoscale', 'Full range'])
style = {'description_width': 'initial'}
R_ptp = widgets.Text(description="V/I ptp (ohms)",style=style)
R_avg = widgets.Text(description="V/I avg (ohms)",style=style)
R_ptp.layout.width = '150px'
R_avg.layout.width = '150px'

update_display()
target = show(column(p1,p2), notebook_handle=True)
display(widgets.HBox([stop_button,scale_mode, R_ptp, R_avg]))

while True:
    ipython.kernel.do_one_iteration()
    update_display()
    if stop_button.value:
        break
    push_notebook(handle=target)

print("Live view has stopped")

t_acq 0.631063 t_tran 0.535053


t_acq 0.651065 t_tran 0.515052
t_acq 0.641064 t_tran 0.386039
t_acq 0.641064 t_tran 0.425043
t_acq 0.648065 t_tran 0.605061
Live view has stopped


# IV Live Display

In [12]:
def update_display():
    ipython.kernel.do_one_iteration()
    I,V,t = get_IV_t()
    ipython.kernel.do_one_iteration()
    I *= 1e6
    V *= 1e6
    source1.data = dict(x=I, y=V)
    
    # Update widgets
    I_avg = np.average(I)
    I_ptp = np.max(I) - np.min(I)
    V_avg = np.average(V)
    V_ptp = np.max(V) - np.min(V)
    widgets = (I_avg_w, V_avg_w,  R_avg_w, I_ptp_w, V_ptp_w, R_ptp_w)
    vals = (I_avg, V_avg, V_avg / I_avg, I_ptp, V_ptp, V_ptp / I_ptp)
    for w,v in zip(widgets,vals):
        w.value = "%.2f" % v
    
    # Update scaling
    if scale_mode.value == "Autoscale":
        p1.x_range.start = np.min(I)-I_ptp*0.1
        p1.x_range.end = np.max(I)+I_ptp*0.1
        p1.y_range.start = np.min(V)-V_ptp*0.1
        p1.y_range.end = np.max(V)+V_ptp*0.1
    else:
        p1.x_range.start = -x_full_scale
        p1.x_range.end = x_full_scale
        p1.y_range.start = -y_full_scale
        p1.y_range.end = y_full_scale
        
    x = np.array([np.min(I), np.max(I)])
    R = line_slope.value
    y = x*R
    source2.data = dict(x=x, y=y)
    
    if plot_line_button.value:
   
        slope_line.visible = True
    else:
        slope_line.visible = False    
        
        
range1 = Range1d()
range2 = Range1d()
source1 = ColumnDataSource()
source2 = ColumnDataSource()

p1 = figure(title="IV", plot_height=500, plot_width=700,toolbar_location=None, x_range = range1, y_range = range2)
p1.yaxis.axis_label = 'Measured Voltage (uV)'
p1.xaxis.axis_label = 'Applied Current (uA)'
p1.line('x', 'y', source=source1)
slope_line = p1.line('x','y',source=source2, color='red')
slope_line.visible = False

x_full_scale = 240 #uA
y_full_scale = 800 # uV

stop_button = widgets.ToggleButton(description='Stop')
scale_mode = widgets.RadioButtons(options=['Autoscale', 'Full range'])
style = {'description_width': 'initial'}

I_avg_w = widgets.Text(description="I_avg (uA)",style=style)
V_avg_w = widgets.Text(description="V_avg (uV)",style=style)
I_ptp_w = widgets.Text(description="I_ptp (uA)",style=style)
V_ptp_w = widgets.Text(description="V_ptp (uV)",style=style)
R_ptp_w = widgets.Text(description="V/I ptp (ohms)",style=style)
R_avg_w  = widgets.Text(description="V/I avg (ohms)",style=style)

plot_line_button = widgets.ToggleButton(description="Plot line")
line_slope = widgets.FloatText(description="R_line (ohms)")
line_slope.value = 50
copy_slope_button = widgets.Button(description="Copy PTP slope")

def copy_slope(w):
    line_slope.value = float(R_ptp_w.value)
copy_slope_button.on_click(copy_slope)

for w in (I_avg_w, V_avg_w, I_ptp_w, V_ptp_w, R_ptp_w, R_avg_w):
    w.layout.width = '150px'

update_display()
target = show(p1, notebook_handle=True)
row1 = widgets.HBox([I_avg_w, V_avg_w, R_avg_w])
row2 = widgets.HBox([I_ptp_w, V_ptp_w, R_ptp_w])
row3 = widgets.HBox([plot_line_button, line_slope, copy_slope_button])
row4 = widgets.HBox([stop_button,scale_mode])
display(widgets.VBox([row1,row2,row3,row4]))

while True:
    update_display()
    if stop_button.value:
        break
    push_notebook(handle=target)

print("Live view stopped")

Live view stopped
